In [1]:
#load path
import os
import sys
import astropy
from astropy.io import fits
from astropy.table import QTable

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.stats import sigma_clipped_stats, sigma_clip

from astropy.wcs import WCS
from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus, ApertureStats
from astropy.stats import SigmaClip

from astropy.visualization import (ImageNormalize, MinMaxInterval,PercentileInterval,SqrtStretch)
import glob

import warnings


In [2]:
path='./data/'
rawpath=r'/Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/'
calibpath='./data/Calibs/'
skyflatspath='./data/Test_skyflats/'
gain=0.25 #e-/ADU at G = 125

In [4]:
#now reduce science images
#in every directory of rawpath, find if there is a LIGHT folder and reduce the images inside
for root, dirs, files in os.walk(rawpath):
    if 'SKYFLAT' in dirs:

        light_dir = os.path.join(root, 'SKYFLAT')
        for filt in ['G','R','I','Ha','Hb','OIII','SII']:

            sciencelist=glob.glob(light_dir+'/*SKYFLAT*{}*.fits'.format(filt))
            if len(sciencelist)==0:
                continue

            skyflats = []

            date = root.split('/')[-1]

            print('Reducing science images for date: {}'.format(date))
            #find closest date in calibpath
            allbias = glob.glob(calibpath+'/MasterBias_*.fits')
            alldates = [os.path.basename(f).split('_')[1].split('.')[0] for f in allbias]
            date_diffs = [abs((astropy.time.Time(date) - astropy.time.Time('{}-{}-{}'.format(d[:4],d[4:6],d[6:8]))).jd) for d in alldates]
            closest_date = alldates[np.argmin(date_diffs)]
            print('Using calibration bias from date: {}-{}-{}'.format(closest_date[:4],closest_date[4:6],closest_date[6:8]))

            masterbias=fits.open(calibpath+'/MasterBias_{}.fits'.format(closest_date))[0].data

            for img in sciencelist:
               print('  Running file {}'.format(img))
               hdu=fits.open(img)
               data=hdu[0].data
               header=hdu[0].header
               filter=header['FILTER']
               exptime=header['EXPTIME']

               #find closest flat for this filter
               allflats = glob.glob(calibpath+'/MasterFlat_{}_*.fits'.format(filter))
               alldates = [os.path.basename(f).split('_')[-1].split('.')[0] for f in allflats]
               date_diffs = [abs((astropy.time.Time(date) - astropy.time.Time('{}-{}-{}'.format(d[:4],d[4:6],d[6:8]))).jd) for d in alldates]
               closest_date = alldates[np.argmin(date_diffs)]
               print('  Using calibration flat from date: {}-{}-{}'.format(closest_date[:4],closest_date[4:6],closest_date[6:8]))

               normflat=fits.open(calibpath+'MasterFlat_{}_{}.fits'.format(filter, closest_date))[0].data

               reduced=((data-masterbias)/normflat)
               reduced/=np.nanmedian(reduced)

               skyflats.append(reduced)

            skyflat = np.median(np.array(skyflats), axis=0)
            hdu = fits.PrimaryHDU(skyflat)
            hdu.writeto(skyflatspath+'/SkyFlat_{}_{}.fits'.format(filt,date), overwrite=True)



Reducing science images for date: 2025-11-28
Using calibration bias from date: 2025-11-27
  Running file /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-11-28/SKYFLAT/2025-11-28_17-15-26_sci_SKYFLAT_G_exp15.00_0000.fits
  Using calibration flat from date: 2025-11-27
  Running file /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-11-28/SKYFLAT/2025-11-28_17-14-42_sci_SKYFLAT_G_exp15.00_0001.fits
  Using calibration flat from date: 2025-11-27
  Running file /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-11-28/SKYFLAT/2025-11-28_17-19-56_sci_SKYFLAT_G_exp35.00_0000.fits
  Using calibration flat from date: 2025-11-27
  Running file /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-11-28/SKYFLAT/2025-11-28_17-18-17_sci_SKYFLAT_G_exp30.00_0000.fits
  Using calibration flat from date: 2025-11-27
  Running file /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-11-28/SKYFLAT/2025-11-28_17-16-10_sci_SKYFLAT_G_e

In [5]:
testfile = rawpath+'/2025-11-29/LIGHT/2025-11-29_21-26-28_sci_PSZ2G114.79-33.71_G_exp300.00_0000.fits'
skyflat = skyflatspath+'/SkyFlat_{}_{}.fits'.format('G','2025-11-28')

hdu=fits.open(testfile)
data=hdu[0].data
header=hdu[0].header

hduskyflat=fits.open(skyflat)
normskyflat=hduskyflat[0].data

reduced = (data-masterbias)/normflat
reduced2 = reduced/normskyflat

hdu1 = fits.PrimaryHDU(reduced, header=header)
hdu2 = fits.ImageHDU(reduced2, header=header)
hdulist = fits.HDUList([hdu1, hdu2])
hdulist.writeto(skyflatspath+'TESTDATA.fits', overwrite=True)
